In [ ]:
import tkinter as tk
from tkinter import messagebox
import speech_recognition as sr
import sqlite3

# Function to clean up the email input (correct speech-to-text errors)
def clean_email(text):
    text = text.lower().replace(" at the rate of ", "@").replace(" at ", "@")
    text = text.replace(" dot ", ".")
    return text

# Function to recognize speech and insert it into the entry
def recognize_speech(entry_field, field_name):
    recognizer = sr.Recognizer()
    recognizer.energy_threshold = 300  # Adjust if speech is too low
    recognizer.pause_threshold = 0.8  # Allow brief pauses

    # Display "Hey, I am listening!" in green at the top when system starts listening
    listening_label.config(text="Hey, I am listening!", fg="green")
    root.update()  # Update the window to reflect the change immediately

    with sr.Microphone() as source:
        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=10)
            text = recognizer.recognize_google(audio)

            # Debugging: Print recognized text
            print(f"Recognized text for {field_name}: {text}")

            # Special processing for email
            if field_name == "Email":
                text = clean_email(text)  # Clean email input from speech mistakes

            # Insert the recognized text into the entry field
            entry_field.delete(0, tk.END)
            entry_field.insert(tk.END, text)

            # Clear the listening label after successful recognition
            listening_label.config(text="")

        except sr.UnknownValueError:
            listening_label.config(text="Sorry, I could not understand the audio.", fg="red")
        except sr.RequestError:
            listening_label.config(text="API Error: Could not request results.", fg="red")
        except sr.WaitTimeoutError:
            listening_label.config(text="Speech input timed out. Please try again.", fg="red")
        except Exception as e:
            listening_label.config(text=f"Error: {str(e)}", fg="red")

# Function to connect to the SQLite database and create the table
def create_database():
    conn = sqlite3.connect('student_admission.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS students (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            age INTEGER,
            gender TEXT,
            course TEXT,
            email TEXT
        )
    ''')
    conn.commit()
    conn.close()

# Function to submit student information
def submit_form():
    name = name_entry.get()
    age = age_entry.get()
    gender = gender_entry.get()
    course = course_entry.get()
    email = email_entry.get()

    # Debugging: Print out the data being submitted
    print(f"Submitting data - Name: {name}, Age: {age}, Gender: {gender}, Course: {course}, Email: {email}")

    if name and age and gender and course and email:
        # Store the data in the database
        try:
            conn = sqlite3.connect('student_admission.db')
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO students (name, age, gender, course, email) 
                VALUES (?, ?, ?, ?, ?)
            ''', (name, age, gender, course, email))
            conn.commit()
            conn.close()

            student_data = f"Name: {name}\nAge: {age}\nGender: {gender}\nCourse: {course}\nEmail: {email}"
            messagebox.showinfo("Form Submitted", student_data)
        except sqlite3.Error as e:
            messagebox.showerror("Database Error", f"An error occurred: {str(e)}")
    else:
        messagebox.showwarning("Input Error", "Please fill in all fields.")

# Function to fetch and display student data
def view_data():
    try:
        conn = sqlite3.connect('student_admission.db')
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM students')
        records = cursor.fetchall()  # Fetch all records

        # Create a new window to display the data
        data_window = tk.Toplevel(root)
        data_window.title("Student Data")
        data_window.geometry("400x300")

        # Create a text widget to display the data
        text_widget = tk.Text(data_window)
        text_widget.pack(expand=True, fill='both')

        # Write the records in a tabular format
        text_widget.insert(tk.END, "ID | Name | Age | Gender | Course | Email\n")
        text_widget.insert(tk.END, "-" * 60 + "\n")
        for row in records:
            text_widget.insert(tk.END, f"{row[0]} | {row[1]} | {row[2]} | {row[3]} | {row[4]} | {row[5]}\n")

        # Close the database connection
        conn.close()

    except sqlite3.Error as e:
        messagebox.showerror("Database Error", f"An error occurred: {str(e)}")

# Creating the tkinter window
root = tk.Tk()
root.title("Student Admission Form")

# Adjust window size if needed
root.geometry("500x350")

# Call the create_database function to set up the database
create_database()

# Listening status label (this will show messages in different colors at the top)
listening_label = tk.Label(root, text="", font=("Helvetica", 10, "bold"))
listening_label.grid(row=0, column=1, padx=10, pady=5)

# Labels and entry fields
tk.Label(root, text="Student Name:").grid(row=1, column=0, padx=10, pady=10, sticky=tk.W)
name_entry = tk.Entry(root, width=30)
name_entry.grid(row=1, column=1, padx=10, pady=10)
name_button = tk.Button(root, text="Speak Name", command=lambda: recognize_speech(name_entry, "Name"))
name_button.grid(row=1, column=2, padx=10, pady=10)

tk.Label(root, text="Age:").grid(row=2, column=0, padx=10, pady=10, sticky=tk.W)
age_entry = tk.Entry(root, width=30)
age_entry.grid(row=2, column=1, padx=10, pady=10)
age_button = tk.Button(root, text="Speak Age", command=lambda: recognize_speech(age_entry, "Age"))
age_button.grid(row=2, column=2, padx=10, pady=10)

tk.Label(root, text="Gender:").grid(row=3, column=0, padx=10, pady=10, sticky=tk.W)
gender_entry = tk.Entry(root, width=30)
gender_entry.grid(row=3, column=1, padx=10, pady=10)
gender_button = tk.Button(root, text="Speak Gender", command=lambda: recognize_speech(gender_entry, "Gender"))
gender_button.grid(row=3, column=2, padx=10, pady=10)

tk.Label(root, text="Course:").grid(row=4, column=0, padx=10, pady=10, sticky=tk.W)
course_entry = tk.Entry(root, width=30)
course_entry.grid(row=4, column=1, padx=10, pady=10)
course_button = tk.Button(root, text="Speak Course", command=lambda: recognize_speech(course_entry, "Course"))
course_button.grid(row=4, column=2, padx=10, pady=10)

tk.Label(root, text="Email:").grid(row=5, column=0, padx=10, pady=10, sticky=tk.W)
email_entry = tk.Entry(root, width=30)
email_entry.grid(row=5, column=1, padx=10, pady=10)
email_button = tk.Button(root, text="Speak Email", command=lambda: recognize_speech(email_entry, "Email"))
email_button.grid(row=5, column=2, padx=10, pady=10)

# Submit and View Data buttons
submit_button = tk.Button(root, text="Submit", command=submit_form)
submit_button.grid(row=6, column=1, padx=10, pady=10)

view_button = tk.Button(root, text="View Student Data", command=view_data)
view_button.grid(row=7, column=1, padx=10, pady=10)

# Start the main loop
root.mainloop()


Recognized text for Name: Dinesh Kumar
Recognized text for Age: 23
Recognized text for Gender: female
Recognized text for Email: Ramesh at the rate of gmail
Submitting data - Name: Ramesh, Age: 23, Gender: Male, Course: EEE, Email: ramesh@gmail.com


In [8]:
import sqlite3

# Function to display all students in a tabular format
def display_students():
    conn = sqlite3.connect('student_admission.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM students')
    rows = cursor.fetchall()

    # Check if there are any rows in the database
    if not rows:
        print("No student data found.")
        conn.close()
        return

    # Print the table headers
    print("{:<5} {:<20} {:<5} {:<10} {:<10} {:<30}".format("ID", "Name", "Age", "Gender", "Course", "Email"))
    print("-" * 85)  # Line to separate headers from data

    # Print each row in a formatted way
    for row in rows:
        print("{:<5} {:<20} {:<5} {:<10} {:<10} {:<30}".format(row[0], row[1], row[2], row[3], row[4], row[5]))

    conn.close()

# Call the function
display_students()


ID    Name                 Age   Gender     Course     Email                         
-------------------------------------------------------------------------------------
1     Dinesh kumar         22    Male       CSM        dinesh@gmail.com              
2     Aditya               19    Male       CSM        adi@gmail.com                 
3     Pushpa               21    Female     CSM        pushpa@gmail.com              
4     Chiranjeevi          22    Male       EEE        ch@gmail.com                  
5     Shreya               23    Female     CSE        shreya@gmail.com              
6     SIndhu               23    Female     CSE        sindhu@gmail.com              
7     Suraj Kumar          22    Male       CSM        suraj@gmail.com               
8     Ramesh               23    Male       EEE        ramesh@gmail.com              
